In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import urllib
from urllib.request import urlopen, urlretrieve
from io import BytesIO
from pyspark.sql import functions as F
from pyspark.sql import types as T
import numpy as np
import sys
import pickle
import pandas as pd
import tensorflow as tf

In [36]:
# to load the dataset
#testfile = urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz","cifar.tar.gz")

In [3]:
def load_CIFAR10_batch(filename):
  '''load data from single CIFAR-10 file'''

  with open(filename, 'rb') as f:
    if sys.version_info[0] < 3:
      dict = pickle.load(f)
    else:
      dict = pickle.load(f, encoding='latin1')
    x = dict['data']
    y = dict['labels']
    x = x.astype(float)
    y = np.array(y)
  return x, y

In [4]:
filename = 'cifar-10-batches-py/data_batch_1'
X, Y = load_CIFAR10_batch(filename)

In [5]:
rdd = zip(X.tolist(),Y.tolist(),np.arange(10000).tolist())
rdd = sc.parallelize(rdd)
df3 = rdd.toDF(['images','labels','index'])
numImages = df3.count()

In [ ]:
#df3 = rdddf.withColumn("index",F.monotonicallyIncreasingId())


In [6]:
batch_size = 10
indices = np.random.choice(numImages,batch_size)
array = (df3.where(F.col("index") == 73).select(F.col("images"))).collect()
array = (df3.where(F.col("index") == int(85))).collect()
#array
#array[0].images

In [7]:
def input_images(indices):
    a = []
    b = []
    for x in indices:
        array = (df3.where(F.col("index") == int(x))).collect()
        a.append(np.array(array[0].images))
        b.append(np.array(array[0].labels))
    return np.array(a), np.array(b)

In [8]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [9]:
batch_size = 100
learning_rate = 0.005
max_steps = 1000


image_placeholders = tf.placeholder(tf.float32, shape=[None,3072])
labels_placeholders = tf.placeholder(tf.int64, shape=[None])

weights = tf.Variable(tf.zeros([3072,10]))
biases = tf.Variable(tf.zeros([10]))

logits = tf.matmul(image_placeholders, weights) + biases

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels_placeholders))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logits,1),labels_placeholders)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(max_steps):
        indices = np.random.choice(10000,batch_size) #number of rows hardcoded(need to change this)
        image_batch,labels_batch = input_images(indices)
        if i%100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={image_placeholders: image_batch,labels_placeholders:labels_batch})
            print('Step {:5d}: training accuracy {:g}'.format(i,train_accuracy))

        sess.run(train_step, feed_dict={image_placeholders: image_batch, labels_placeholders:labels_batch})
        
    test_accuracy = sess.run(accuracy, feed_dict={image_placeholders: image_batch,labels_placeholders: image_batch})
    print('Test accuracy {:g}'.format(test_accuracy))

Step     0: training accuracy 0.09
